<a href="https://colab.research.google.com/github/pgurazada/reel-framer/blob/main/reel_framer_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 43.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.23-cp310-cp310-manylinux_2_35_x86_64.whl size=8162245 sha256=789bb7d46def8373b42a229dd9afb3d8b9bc5e935572a5355e5de8b709977821
  Stored in directory: /root/.cache/pip/wheels/8b/7c/23/5851b51f3b9088d6f7a5ba6d27b3494aa4940bf291b43ee04d
Successfully built llama-cpp-python


In [4]:
!pip install -q huggingface_hub

In [25]:
import json
import re

from json import JSONDecodeError

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [6]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

In [7]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

The python interface to Llama CPP allows us to load the model binary (*.gguf) on a combination of CPU and GPU and provides an Open AI compatible serving interface.

In [8]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192 # Context window
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


The general Mistral template for prompting is presented below (start of sentence token( `<s>`), instruction tokens (`[INST]`) and new lines (`\n`) are important and should not be skipped in the template).

In [9]:
mistral_prompt_template = """<s>[INST]
{system_message}

{user_message} [/INST]"""

In [18]:
def get_json_info(info_row: str) -> dict:
    """
    Fixes malformed JSON outputs from LLMs

    """
    try:
        info = json.loads(info_row)
    except JSONDecodeError:
        data = {
        }
        try:

            for s in info_row.split('","'):
                if not s:
                    continue
                key, val = s.split(":", maxsplit=1)
                key = key.strip().lstrip("{").strip('"')
                val: str = re.sub('"', '\\"', val.lstrip('"').strip('\"}'))
                data[key] = val
        except ValueError:
            print("ERROR:", info_row)
        info = data
    return info

# Step 1: Extract News Information

In [10]:
news_extractor_system_message = """
You are tasked to extract key information from news articles.
You will be presented a news article that begins with ###News Article.

Instructions:
Extract the following items from the news article in the input in a JSON format:
{
    news setting: <Where did this news event take place?>,
    news characters and their main activities: <List names of up to five main \
stakeholders in this news event and what they mainly did.>
    news plot summary: <What happened in the news event?>
    news information points: <What are the three most important things in this news story?>
    news plot elements: <What are the four main plot points of the news story?>
}
To reiterate, your answer should be in the JSON format specified above.
"""

In [11]:
news_user_message_template = """###News Article \n{news_article}"""

In [12]:
news_article_input = """
The US Food and Drug Administration (USFDA) last week approved two breakthrough gene therapies — Casgevy by Vertex Pharmaceuticals and CRISPR Therapeutics, and Lyfgenia by Bluebird Bio — for sickle cell disease (SCD) in patients 12 years and older.

The development marks a milestone medical advancement in treating a debilitating disease that primarily affects red blood cells’ capacity to carry adequate oxygen across the body, with the use of innovative cell-based gene therapies.
Both approved products are made from patients’ own blood stem cells, which are modified, and are given back as a one-time, single-dose infusion as part of a hematopoietic (blood) stem cell transplant.

Casgevy utilises CRISPR/Cas9 (Clustered Regularly Interspaced Short Palindromic Repeats-CRISPR associated) technology, a type of genome editing system.
Emmanuelle Charpentier and Jennifer Doudna were awarded the Nobel Prize in Chemistry in 2020 for discovering CRISPR/Cas9 genetic scissors, called one of the gene technology’s sharpest tools.

In India, which has the highest number of SCD carriers in the world, scientists associated with the Council for Scientific and Industrial Research-Institute of Genomics and Integrative Biology (CSIR-IGIB) have been working since 2018 to develop a gene therapy for SCD using the same technology.

“After showing proof of the therapy developed in human-induced pluripotent stem cells (a particular potent type of stem cell that normally only exists during early embryonic development), we are now in preclinical stage of the therapy’s trial,” Debojyoti Chakraborty, lead scientist of the project at CSIR-IGIB, told ThePrint.

The next step after the animal study, he said, is to start a phase-1 clinical trial for SCD patients in India, in partnership with the All India Institute of Medical Sciences (AIIMS) in Delhi and the department of science and technology after regulatory approvals for the therapy are in place.
Once available in the country, the therapy can be a boon for millions of SCD patients in India which this year saw the launch of the National Sickle Cell Anaemia Elimination Mission — targeting to eliminate the disease by 2047.
"""

In [13]:
prompt_for_news_extraction = mistral_prompt_template.format(
    system_message=news_extractor_system_message,
    user_message=news_user_message_template.format(
        news_article=news_article_input
    )
)

In [14]:
response = lcpp_llm(
    prompt=prompt_for_news_extraction,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False
)

In [19]:
news_extraction_output = get_json_info(response["choices"][0]["text"])

In [20]:
news_extraction_output.keys()

dict_keys(['news setting', 'news characters and their main activities', 'news plot summary', 'news information points', 'news plot elements'])

# Step 2: Create a comedic analogy

In [39]:
comedic_analogy_prompt_template = """
You are tasked to create a comedic analogy based on key information extracted from a news article.

Instructions:
1. List three unique comedic analogies for the situation in the following story:
{news_plot_summary}. Incorporate the following characters only: {news_characters_and_their_main_activities}.
2. Decide the main characters of the news event as two of the most dominant characters in the summary: {news_plot_summary}
3. To act out this analogous premise use the location mentioned here: {news_setting}

Return your output as a JSON with the three analogies as keys like so:
- Comedic Analogy 1: <first analogy>,
- Comedic Analogy 2: <second analogy>,
- Comedic Analogy 3: <third analogy>
"""

In [40]:
prompt_for_comedic_analogy = comedic_analogy_prompt_template.format(
    news_plot_summary=news_extraction_output['news plot summary'],
    news_characters_and_their_main_activities=news_extraction_output['news characters and their main activities'],
    news_setting=news_extraction_output['news setting']
)

In [41]:
mistral_prediction_template = """<s>[INST]{user_message}[/INST]"""

In [42]:
response = lcpp_llm(
    prompt=mistral_prediction_template.format(
        user_message=prompt_for_comedic_analogy
    ),
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False
)

Llama.generate: prefix-match hit


In [44]:
comedic_analogies = get_json_info(response["choices"][0]["text"])

In [45]:
comedic_analogies.keys()

dict_keys(['Comedic Analogy 1', 'Comedic Analogy 2', 'Comedic Analogy 3'])

# Step 3: Create a script

In [46]:
comedic_script_prompt_template = """
Write a script for a comedy skit about: {script_plot}. Cover the following information: {news_information_points}.
The characters should be exactly: {news_characters_and_their_main_activities}.
It should be set in {news_setting}. It should be entertaining.
The dialogue should be colloquial and engaging. The dialogue should be 10 to 12 lines long.
Each line of dialogue should be short - less than 20 words. End it with a punchline.
"""

In [47]:
prompt_for_comedic_script = comedic_script_prompt_template.format(
    script_plot=comedic_analogies['Comedic Analogy 1'],
    news_information_points=news_extraction_output['news information points'],
    news_characters_and_their_main_activities=news_extraction_output['news characters and their main activities'],
    news_setting=news_extraction_output['news setting']
)

In [48]:
mistral_prediction_template = """<s>[INST]{user_message}[/INST]"""

In [49]:
response = lcpp_llm(
    prompt=mistral_prediction_template.format(
        user_message=prompt_for_comedic_script
    ),
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False
)

Llama.generate: prefix-match hit


In [50]:
print(response["choices"][0]["text"])

 (Scene opens at the FDA headquarters, where Emmanuelle Charpentier and Jennifer Doudna are excitedly welcoming Vertex Pharmaceuticals and CRISPR Therapeutics)

Emmanuelle: "Hey there, long time no see! Welcome to your new home, Casgevy and CRISPR team!" (Hands them a housewarming gift)

Jennifer: "Thanks Emmanuelle, we're thrilled to finally move in! And hey Bluebird Bio, nice to see you too!"

(Bluebird Bio enters with Lyfgenia in hand)

Debojyoti (excitedly): "Hello everyone, Deb from CSIR-IGIB here. I heard there's a new kid on the block! Hopefully AIIMS invites me to the next housewarming party."

Vertex: "Absolutely, Deb! We'll make sure you get an invite for your upcoming phase-1 trials!"

Emmanuelle: (Laughs) "Well this is getting competitive. I guess we better start planning some FDA gene therapy cookouts then!"

Jennifer: "You got it, Emmanuelle! And remember, let's make sure everyone plays nice and uses CRISPR/Cas9 technology correctly."

Debojyoti: "Agreed! No gene-editing 